In [52]:
from sympy import Add, Mul, symbols, sympify, I, ccode, expand, diff, simplify


we could make it so that an array represents a 'pathway' of operation and then is procedurally operated through in the opencl kernel. this would allow functions to be passed in without recompiling the kernel. would likely lead to optimization loss

for now we are using a method in which c++ is generated and compiled in along with helper functions that allow it to process the injected mathematics.

maybe a compromise between the two methods could be found?

In [53]:
def compute_derivatives(expr_str, var='z', real_var='x', imag_var='y', param='w'):
    x, y, a = symbols(f'{real_var} {imag_var} {param}', real=True)
    z = x + I*y
    expr = sympify(expr_str)
    expr = expr.subs(var, z)
    expr_expanded = expand(expr)
    f_real = expr_expanded.as_real_imag()[0]
    f_imag = expr_expanded.as_real_imag()[1]

    f_prime_real_x = diff(f_real, x)
    f_prime_real_y = diff(f_real, y)
    f_prime_imag_x = diff(f_imag, x)
    f_prime_imag_y = diff(f_imag, y)

    return f_real, f_imag, f_prime_real_x, f_prime_real_y, f_prime_imag_x, f_prime_imag_y

#introduces I
f = "(cos(sin(I/z))*z*z*z-1)/(cos(sin(z*z*z*z-1)))"

f = "z*z*z-1"

f_real, f_imag, f_prime_real_x, f_prime_real_y, f_prime_imag_x, f_prime_imag_y = compute_derivatives(f)
for q in f_real, f_imag, f_prime_real_x, f_prime_real_y, f_prime_imag_x, f_prime_imag_y:
    print(q)

x**3 - 3*x*y**2 - 1
3*x**2*y - y**3
3*x**2 - 3*y**2
-6*x*y
6*x*y
3*x**2 - 3*y**2


In [54]:
def handle_complex_operations(expr, depth=0):
    """
    Recursively handle complex operations by breaking them down into equivalent sequences 
    of real arithmetic operations and calls to helper functions.
    """
    x, y = symbols('x y', real=True)
    z = x + I*y

    # Base case: if the expression is atomic (i.e., cannot be broken down further)
    if expr.is_Atom:
        return ccode(expr).replace("I", "1")  # Replacing I representation

    # Recursive case
    refined_parts = []
    for part in expr.args:
        refined_parts.append(handle_complex_operations(part, depth + 1))

    # Replacement with helper functions
    if expr.func == Add:
        # Handle addition
        real_parts = [part for part in refined_parts if "I" not in part]
        imag_parts = [part.replace('*I', '') for part in refined_parts if "I" in part]
        
        real_str = ' + '.join(real_parts) if real_parts else '0'
        imag_str = ' + '.join(imag_parts) if imag_parts else '0'
        return f"complex_add_real({real_str}, {imag_str}, 0, 0)"
    
    elif expr.func == Mul:
        # Handle multiplication
        real_parts = [part for part in refined_parts if "I" not in part]
        imag_parts = [part.replace('*I', '') for part in refined_parts if "I" in part]
        
        real_str = ' * '.join(real_parts) if real_parts else '1'
        imag_str = ' * '.join(imag_parts) if imag_parts else '1'
        
        return (f"complex_mul_real({real_str}, {imag_str}, 0, 0), "
                f"complex_mul_imag({real_str}, {imag_str}, 0, 0)")
    
    elif expr.func == Pow and '-1' in ccode(expr.args[1]):
        # Handle division
        numerator = refined_parts[0]
        denominator = refined_parts[1].replace('pow(', '').replace(', -1)', '')
        
        return (f"complex_div_real({numerator}, 0, {denominator}, 0), "
                f"complex_div_imag({numerator}, 0, {denominator}, 0)")
    
    else:
        # For other operations, just join the parts with the function name
        return f"{expr.func.__name__}({', '.join(refined_parts)})"

for q in f_real, f_imag, f_prime_real_x, f_prime_real_y, f_prime_imag_x, f_prime_imag_y:
    print(handle_complex_operations(q))


complex_add_real(-1 + Pow(x, 3) + complex_mul_real(-3 * x * Pow(y, 2), 1, 0, 0), complex_mul_imag(-3 * x * Pow(y, 2), 1, 0, 0), 0, 0, 0)
complex_add_real(complex_mul_real(-1 * Pow(y, 3), 1, 0, 0), complex_mul_imag(-1 * Pow(y, 3), 1, 0, 0) + complex_mul_real(3 * y * Pow(x, 2), 1, 0, 0), complex_mul_imag(3 * y * Pow(x, 2), 1, 0, 0), 0, 0, 0)
complex_add_real(complex_mul_real(-3 * Pow(y, 2), 1, 0, 0), complex_mul_imag(-3 * Pow(y, 2), 1, 0, 0) + complex_mul_real(3 * Pow(x, 2), 1, 0, 0), complex_mul_imag(3 * Pow(x, 2), 1, 0, 0), 0, 0, 0)
complex_mul_real(-6 * x * y, 1, 0, 0), complex_mul_imag(-6 * x * y, 1, 0, 0)
complex_mul_real(6 * x * y, 1, 0, 0), complex_mul_imag(6 * x * y, 1, 0, 0)
complex_add_real(complex_mul_real(-3 * Pow(y, 2), 1, 0, 0), complex_mul_imag(-3 * Pow(y, 2), 1, 0, 0) + complex_mul_real(3 * Pow(x, 2), 1, 0, 0), complex_mul_imag(3 * Pow(x, 2), 1, 0, 0), 0, 0, 0)
